In [1]:
from imagery_psychophysics.src.variational import *
from imagery_psychophysics.utils.variational_utils import open_imagery_probe_data
from imagery_psychophysics.src.stirling_maps import stirling_num_of_2nd_kind as snk
from imagery_psychophysics.src.stirling_maps import stirling_partitions
from skimage.morphology import binary_dilation, binary_erosion
import numpy as np
import pandas as pd
import math
import copy
from matplotlib import pyplot as plt
from os.path import join
from PIL.Image import open as open_image
from mpl_toolkits.axes_grid1 import ImageGrid
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline

Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
/tmp/try_flags_HuTFt5.c:4:19: fatal error: cudnn.h: No such file or directory
compilation terminated.

Mapped name None to device cuda: GeForce GTX TITAN X (0000:01:00.0)


In [2]:
resultsDict = dict()
experiment_df = open_imagery_probe_data()

In [3]:
experiment_df['image'].unique()

array(['candle_01', 'peaches_05', 'cheeseburger_03', 'grape_juice_02'],
      dtype=object)

In [ ]:
for subject, subject_group in experiment_df.groupby('subj'):
    resultsDict[subject] = dict()
    for state, state_group in subject_group.groupby('state'):
        resultsDict[subject][state] = dict()
        for targetImageName, target_group in state_group.groupby('image'):
            ##get data
            windows, resp, _, targetObjectMap,targetImage = open_imagery_probe_data(subject, state, targetImageName)
            targetObjectMap = targetObjectMap.resize(targetImage.size) ##have to do this because not matched in size
            
            ##==========construct model random variables
            ##number of objects
            nObj = numObjects()

            ##dispersion on category prior: here we set a hyperprior
            pDisp = priorDispersion()
            dispersion = 1.0
            pDisp.set_value(dispersion)

            ##resolution of object map Z
            nPixels = numPixels()

            ##category prior and object map
            catProb = categoryProbs(nObj,pDisp)
            Z = latentObjMap(catProb,nPixels)

            ##noise params
            nP = noiseParams()

            ##windows: we change their shape a little to make them easier to work with
            desiredWindowShape = (375,600)
            workingScale = 5 ##(50,80)
            w = probes()
            ##this whole "resolutions" thing doesn't work!! not a bug, just 90% useless as a tool for selecting
            ##a reasonable resolution for the model
            resolutions, workingResolution = w.resolve(desiredWindowShape, workingScale)
            w.set_value(w.reshape(windows, workingResolution),flatten=True)
            print 'working resolution is (%d, %d)' %(workingResolution[0], workingResolution[1])


            ##response object
            r = responses(Z,nP)

            ##fake data
            r.set_values(windows=w)

            r.set_values(data=resp)

            print 'total observations: %d' %(r.N)

            tMap = target_image()
            targetObjectMap_test, targetImage_test = tMap.reshape(targetObjectMap,workingResolution,targetImage=targetImage)
            tMap.set_values(targetObjectMap_test, targetImage_test)
            
            ##==gonna cheat here and set the number of objects using target map
            thisK = tMap.numberTargetObjects
            r.Z.categoryPrior.numObjects.set_value(tMap.numberTargetObjects)
            
            ##==fixing resolution at some reasonble thing
            ##==resolutions[1] = (10 x 16)
            thisNumPix = (resolutions[1][0], resolutions[1][1])
            ##==or just choose (25 x 40) because it maps up to workingScale, which is (50,80)
            thisNumPix = (25,40)
            r.Z.numPixels.set_value(thisNumPix[0], thisNumPix[1])
            
            
            ##=============
            
            ##instantiate the variational inferences we want to perform
            iqZ = inferQZ()
            iqPi = inferQPi()

            ##...and the parameter optimizations (point-estimate) we want
            oNP = optimizeNoiseParams()

            ##variational inference combines them all together
            vi = VI(r, iqZ,oNP, iqPi)

            ### Run variational inference

            ##inference algorithm parameters
            initialNoisinessOfZ = 0.2
            pOn_init, pOff_init = .8, 0.2
            densityOfNoiseParamGrid = 50
            numStarterMaps = 20
            numSamplesForComputingObjectCountProbs = 20
            maxNumIterations = 300
            trainTestSplit = 0.8
            trainRegSplit = .8

            print '=========================(subject, state, target) = (%s, %s, %s) ====' %(subject, state, targetImageName)
            print '=========================(number of object = %d, number of pixels = (%d,%d) )' %(thisK, thisNumPix[0],thisNumPix[1])
            bestModel,_ = vi.run_VI(initialNoisinessOfZ, \
                                 pOn_init, pOff_init, \
                                 densityOfNoiseParamGrid, \
                                 numStarterMaps, \
                                 numSamplesForComputingObjectCountProbs, \
                                 maxNumIterations, \
                                 trainTestSplit, trainRegSplit, \
                                 optimizeHyperParams=False)

            resultsDict[subject][state][targetImageName]= copy.deepcopy(bestModel)

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/expressions.py:179: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


working resolution is (50, 80)
total observations: 345
=========================(subject, state, target) = (AS, img, candle_01) ====
=========================(number of object = 4, number of pixels = (25,40) )
!new best!
ELBO: 107.799896
goodness of fit: -450.638214
posterior_entropy: 558.438110
mean log of predictive distribution over test samples: -2.175599
percent correct over test samples: 23.636364


!new best!
ELBO: 1027.886353
goodness of fit: -279.918091
posterior_entropy: 1307.804443
mean log of predictive distribution over test samples: -1.217792
percent correct over test samples: 40.000000


ELBO: 1027.943848
goodness of fit: -280.336975
posterior_entropy: 1308.280762
mean log of predictive distribution over test samples: -1.219515
percent correct over test samples: 40.000000


ELBO: 1027.092041
goodness of fit: -280.355896
posterior_entropy: 1307.447998
mean log of predictive distribution over test samples: -1.223329
percent correct over test samples: 40.000000


ELBO: 1026

ELBO: 964.559448
goodness of fit: -278.580933
posterior_entropy: 1243.140381
mean log of predictive distribution over test samples: -1.207832
percent correct over test samples: 40.000000


ELBO: 961.294922
goodness of fit: -278.559082
posterior_entropy: 1239.854004
mean log of predictive distribution over test samples: -1.205454
percent correct over test samples: 40.000000


ELBO: 957.056396
goodness of fit: -279.296661
posterior_entropy: 1236.353027
mean log of predictive distribution over test samples: -1.206688
percent correct over test samples: 40.000000


ELBO: 954.405884
goodness of fit: -278.267792
posterior_entropy: 1232.673706
mean log of predictive distribution over test samples: -1.198632
percent correct over test samples: 40.000000


ELBO: 950.526306
goodness of fit: -278.119202
posterior_entropy: 1228.645508
mean log of predictive distribution over test samples: -1.202763
percent correct over test samples: 40.000000


ELBO: 945.876282
goodness of fit: -278.055237
posterior

ELBO: 827.836975
goodness of fit: -251.755554
posterior_entropy: 1079.592529
mean log of predictive distribution over test samples: -1.101928
percent correct over test samples: 40.000000


ELBO: 823.049500
goodness of fit: -254.013840
posterior_entropy: 1077.063354
mean log of predictive distribution over test samples: -1.093249
percent correct over test samples: 40.000000


ELBO: 822.655762
goodness of fit: -254.043930
posterior_entropy: 1076.699707
mean log of predictive distribution over test samples: -1.099019
percent correct over test samples: 40.000000


ELBO: 823.075317
goodness of fit: -253.663834
posterior_entropy: 1076.739136
mean log of predictive distribution over test samples: -1.082627
percent correct over test samples: 40.000000


ELBO: 823.925659
goodness of fit: -251.628418
posterior_entropy: 1075.554077
mean log of predictive distribution over test samples: -1.108824
percent correct over test samples: 43.636364


ELBO: 817.751526
goodness of fit: -253.218918
posterior

ELBO: 787.795349
goodness of fit: -255.247116
posterior_entropy: 1043.042480
mean log of predictive distribution over test samples: -1.098731
percent correct over test samples: 43.636364


ELBO: 789.862183
goodness of fit: -251.169800
posterior_entropy: 1041.031982
mean log of predictive distribution over test samples: -1.098726
percent correct over test samples: 49.090909


ELBO: 786.322571
goodness of fit: -253.501297
posterior_entropy: 1039.823853
mean log of predictive distribution over test samples: -1.090822
percent correct over test samples: 47.272727


ELBO: 782.929565
goodness of fit: -257.174438
posterior_entropy: 1040.104004
mean log of predictive distribution over test samples: -1.100188
percent correct over test samples: 40.000000


ELBO: 789.115479
goodness of fit: -251.339874
posterior_entropy: 1040.455322
mean log of predictive distribution over test samples: -1.093982
percent correct over test samples: 49.090909


ELBO: 787.769226
goodness of fit: -251.478439
posterior

...

ELBO: 771.113403
goodness of fit: -254.315338
posterior_entropy: 1025.428711
mean log of predictive distribution over test samples: -1.104652
percent correct over test samples: 65.454545


ELBO: 773.999268
goodness of fit: -252.550659
posterior_entropy: 1026.549927
mean log of predictive distribution over test samples: -1.116201
percent correct over test samples: 63.636364


ELBO: 773.479492
goodness of fit: -252.452011
posterior_entropy: 1025.931519
mean log of predictive distribution over test samples: -1.112465
percent correct over test samples: 63.636364


ELBO: 770.635742
goodness of fit: -253.926773
posterior_entropy: 1024.562500
mean log of predictive distribution over test samples: -1.106164
percent correct over test samples: 58.181818


ELBO: 772.592163
goodness of fit: -252.543671
posterior_entropy: 1025.135864
mean log of predictive distribution over test samples: -1.113523
percent correct over test samples: 65.454545


ELBO: 767.052673
goodness of fit: -250.712158
posterior

ELBO: 971.211304
goodness of fit: -300.540649
posterior_entropy: 1271.751953
mean log of predictive distribution over test samples: -1.298101
percent correct over test samples: 34.545455


ELBO: 969.081848
goodness of fit: -300.586609
posterior_entropy: 1269.668457
mean log of predictive distribution over test samples: -1.293742
percent correct over test samples: 34.545455


ELBO: 967.007935
goodness of fit: -300.493988
posterior_entropy: 1267.501953
mean log of predictive distribution over test samples: -1.297186
percent correct over test samples: 34.545455


ELBO: 964.738220
goodness of fit: -300.344543
posterior_entropy: 1265.082764
mean log of predictive distribution over test samples: -1.295318
percent correct over test samples: 34.545455


ELBO: 962.257629
goodness of fit: -300.317566
posterior_entropy: 1262.575195
mean log of predictive distribution over test samples: -1.296199
percent correct over test samples: 34.545455


ELBO: 959.283447
goodness of fit: -300.698761
posterior

ELBO: 725.393677
goodness of fit: -258.813599
posterior_entropy: 984.207275
mean log of predictive distribution over test samples: -1.117420
percent correct over test samples: 43.636364


ELBO: 720.909790
goodness of fit: -260.084137
posterior_entropy: 980.993958
mean log of predictive distribution over test samples: -1.118386
percent correct over test samples: 41.818182


ELBO: 719.132812
goodness of fit: -258.203491
posterior_entropy: 977.336304
mean log of predictive distribution over test samples: -1.114057
percent correct over test samples: 43.636364


ELBO: 716.139893
goodness of fit: -258.671509
posterior_entropy: 974.811401
mean log of predictive distribution over test samples: -1.123026
percent correct over test samples: 41.818182


ELBO: 712.682251
goodness of fit: -258.854187
posterior_entropy: 971.536438
mean log of predictive distribution over test samples: -1.117234
percent correct over test samples: 41.818182


ELBO: 708.695312
goodness of fit: -258.976013
posterior_entr

ELBO: 488.531464
goodness of fit: -191.456573
posterior_entropy: 679.988037
mean log of predictive distribution over test samples: -0.842711
percent correct over test samples: 52.727273


ELBO: 487.520203
goodness of fit: -192.181107
posterior_entropy: 679.701294
mean log of predictive distribution over test samples: -0.847294
percent correct over test samples: 52.727273


ELBO: 486.191711
goodness of fit: -193.863464
posterior_entropy: 680.055176
mean log of predictive distribution over test samples: -0.839526
percent correct over test samples: 52.727273


ELBO: 489.870239
goodness of fit: -190.609665
posterior_entropy: 680.479919
mean log of predictive distribution over test samples: -0.845614
percent correct over test samples: 52.727273


ELBO: 487.472870
goodness of fit: -193.072662
posterior_entropy: 680.545532
mean log of predictive distribution over test samples: -0.845788
percent correct over test samples: 52.727273


ELBO: 489.050232
goodness of fit: -191.133362
posterior_entr

ELBO: 484.998932
goodness of fit: -192.474091
posterior_entropy: 677.473022
mean log of predictive distribution over test samples: -0.836707
percent correct over test samples: 52.727273


ELBO: 485.453186
goodness of fit: -192.600998
posterior_entropy: 678.054199
mean log of predictive distribution over test samples: -0.839791
percent correct over test samples: 52.727273


ELBO: 485.313660
goodness of fit: -192.777405
posterior_entropy: 678.091064
mean log of predictive distribution over test samples: -0.842455
percent correct over test samples: 52.727273


ELBO: 485.435059
goodness of fit: -192.050034
posterior_entropy: 677.485107
mean log of predictive distribution over test samples: -0.844440
percent correct over test samples: 52.727273


ELBO: 484.974609
goodness of fit: -192.440842
posterior_entropy: 677.415466
mean log of predictive distribution over test samples: -0.841195
percent correct over test samples: 52.727273


ELBO: 485.900604
goodness of fit: -191.637909
posterior_entr

#### Store the results

In [ ]:
modelList = []
for sbj,subjectDict in resultsDict.iteritems():
    for st, stateDict in subjectDict.iteritems():
        for targ, modelInstance in stateDict.iteritems():
            modelList.append([sbj,st,targ, modelInstance])

In [ ]:
modelDf  = pd.DataFrame(data=modelList, columns=['subject', 'state', 'target', 'model'])
modelDf

In [ ]:
store = pd.io.pytables.HDFStore('/home/tnaselar/FAST/imagery_psychophysics/multi_poly_probes/analysis/all_subjects_(20x32).h5')

In [ ]:
store['models'] = modelDf

In [ ]:
store.close()